In [1]:
# %pip install --upgrade datasets groq

In [6]:
import requests
import os
from datasets import load_dataset

# Config

In [7]:
INPUT_DATASET = "dariolopez/justicio-BOE-A-1978-31229-constitucion-by-articles-qa"

In [8]:
SYSTEM_PROMPT = """
Como un experto en derecho y leyes españolas, tu tarea es responder preguntas sobre el Boletín Oficial del Estado (BOE) de España. Para ello, debes tener en cuenta y utilizar el contexto proporcionado para responder de forma precisa a la pregunta del usuario.
Asegúrate de responder siempre en español. Si no conoces la respuesta o no tienes suficiente información para responderla, simplemente admítelo; no intentes inventar una respuesta.
Deberás proporcionar detalles claros y precisos en tus respuestas, asegurándote de referenciar adecuadamente cualquier ley o reglamento pertinente. Tu objetivo es proporcionar respuestas útiles y precisas para ayudar a los usuarios a entender mejor el BOE y cómo se aplica a sus preguntas.
"""

In [9]:
SYSTEM_PROMPT_CONTEXT = """
El contexto tiene un formato de lista, donde cada elemento será un diccionario con dos claves:
[{'context': 'contexto necesario para contestar la pregunta', 'score': 0.8}]
La clave 'context' contendrá la información necesaria para contestar a la pregunta y la clave 'score' será una puntuación de entre 0.0 y 1.0. Deberás dar más importancia al contexto cuanto mayor sea el score.
En la respuesta no menciones nada sobre el contexto o los scores.
"""

In [10]:
URL = "http://95.217.231.93:5002/semantic_search"

In [11]:
dataset = load_dataset(INPUT_DATASET)

In [13]:
from groq import Groq
import json
import time


client = Groq(
    api_key=os.environ.get('GROQ_API_KEY'),
)

In [14]:
answers_groq = []
contexts = []

for idx, row in enumerate(dataset['train']):
    time.sleep(20)  # groq limits: https://console.groq.com/settings/limits
    try:
        docs = requests.get(URL, params={'input_query': row['question'], 'collection_name': 'boe-multilingual-e5-large'}, timeout=1)
        context_preprocessed = [{"context": doc[0]['page_content'], "score": doc[1]} for doc in docs.json()]
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "system",
                "content": SYSTEM_PROMPT_CONTEXT,
            },
            {"role": "system", "content": "A continuación se proporciona el contexto:"},
            {"role": "system", "content": str(context_preprocessed)},
            {
                "role": "system",
                "content": "A continuación se proporciona la pregunta del usuario:",
            },
            {"role": "user", "content": row['question']},
        ]
        chat_completion = client.chat.completions.create(
            messages=messages,
            model="llama3-70b-8192",
            temperature=0,
            max_tokens=2048,
            stream=False,
        )
        answer = chat_completion.choices[0].message.content
    
    except Exception as e:
        print(e)
        answer = None

    answers_groq.append(answer)
    contexts.append(context_preprocessed)
    print(f"{idx+1} - {row['question']}")
    print(row['answer'])
    print(answer)
    print("\n\n")

1 - ¿Cuáles son los valores superiores del ordenamiento jurídico en España?
La libertad, la justicia, la igualdad y el pluralismo político.
Según la Constitución Española, los valores superiores del ordenamiento jurídico en España son la igualdad, la justicia, la libertad y el pluralismo político. Estos valores se encuentran establecidos en el artículo 1.1 de la Constitución Española, que establece que "España es un Estado social y democrático de Derecho, que propugna como valores superiores de su ordenamiento jurídico la libertad, la justicia, la igualdad y el pluralismo político".



2 - ¿Quién es el titular de la soberanía nacional en España?
El pueblo español.
Según el artículo 1.2 de la Constitución Española, el titular de la soberanía nacional en España es el pueblo español. Esta soberanía es exclusiva e indivisible, y reside únicamente en el conjunto del pueblo español.



3 - ¿Cuál es la forma política del Estado español?
La Monarquía parlamentaria.
Según el artículo 1.3 de la 

In [20]:
len(answers_groq)

515

In [21]:
len(contexts)

515

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['number', 'context', 'question', 'answer', 'context_qa', 'response_groq_llama3_70b_8192'],
        num_rows: 515
    })
})

In [23]:
dataset['train'] = dataset['train'].add_column('context_qa', contexts)
dataset['train'] = dataset['train'].add_column('response_groq_llama3_70b_8192', answers_groq)

ValueError: The table can't have duplicated columns but columns ['context_qa'] are duplicated.

In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['number', 'context', 'question', 'answer', 'context_qa', 'response_groq_llama3_70b_8192'],
        num_rows: 515
    })
})

In [29]:
import huggingface_hub

huggingface_hub.login()

In [31]:
dataset.push_to_hub(f"{INPUT_DATASET}-multilingual-e5-large-groq_llama3_70b_8192")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dariolopez/justicio-BOE-A-1978-31229-constitucion-by-articles-qa-multilingual-e5-large-groq_llama3_70b_8192/commit/bd48923348137911bfd7c4ba32db6bac5a2b6325', commit_message='Upload dataset', commit_description='', oid='bd48923348137911bfd7c4ba32db6bac5a2b6325', pr_url=None, pr_revision=None, pr_num=None)